In [2]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

from tensorboardX import SummaryWriter      

import torchvision
import torchvision.transforms as transforms

from models import *

global best_prec
use_gpu = torch.cuda.is_available()
# print('=> Building model...')
    
    
batch_size = 128
model_name = "VGG16_quant"
model = VGG16_quant()
# print(model.features)
# device = torch.device("cuda") 

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model.classifier = nn.Linear(16, 10) # Fix classifier size to match checkpoint

PATH = "result/mimi_VGG16_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 8966/10000 (90%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
i = 0

for layer in model.modules():
    i = i+1
    if isinstance(layer, QuantConv2d):
        print(i,"-th layer prehooked")
        layer.register_forward_pre_hook(save_output)             
####################################################

dataiter = iter(testloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)

3 -th layer prehooked
7 -th layer prehooked
12 -th layer prehooked
16 -th layer prehooked
21 -th layer prehooked
25 -th layer prehooked
29 -th layer prehooked
34 -th layer prehooked
38 -th layer prehooked
41 -th layer prehooked
46 -th layer prehooked
50 -th layer prehooked
54 -th layer prehooked


In [5]:
weight_q = model.features[27].weight_q
w_alpha = model.features[27].weight_quant.wgt_alpha

# print(weight_q)
w_bit = 2

weight_int = weight_q / (w_alpha / (2**(w_bit-1)-1))
# print(weight_int)

In [6]:
k = 8  # input to your 8→8 conv layer

act = save_output.outputs[k][0]      # [128, 8, 4, 4]
act_alpha = model.features[27].act_alpha
act_bit = 2
act_quant_fn = act_quantization(act_bit)

# quantize + convert to integer
act_q   = act_quant_fn(act, act_alpha)
act_int = act_q / (act_alpha / (2**act_bit - 1))

# sanity check
print(act_int.shape)  # should be [128, 16, 4, 4]


torch.Size([128, 16, 4, 4])


In [7]:
conv_int = torch.nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1, bias=False)
conv_int.weight = torch.nn.Parameter(weight_int.float())

output_int = conv_int(act_int.float())

output_recovered = output_int * (act_alpha / (2**act_bit - 1)) * (w_alpha / (2**(w_bit-1) - 1))

print(output_recovered.shape)

torch.Size([128, 16, 4, 4])


In [8]:
device = act.device

model = model.to(device)

with torch.no_grad():
    conv_ref = model.features[27]
    output_ref = conv_ref(act.float())

err = (output_ref - output_recovered).abs().mean()
print("mean error:", err.item())

mean error: 3.0168860121193575e-07


In [9]:
# act_int.size = torch.Size([128, 64, 32, 32])  <- batch_size, input_ch, ni, nj
a_int = act_int[0,:,:,:]  # pick only one input out of batch

w_int = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))  # merge ki, kj index to kij
 # weight_int: [out_ch, in_ch, ki, kj]  ->  [out_ch, in_ch, kij]
print("w_int shape:", w_int.shape)   # should be [16, 16, 9]
# print(a_int.shape)
padding = 1
stride = 1
array_size = 8 # row and column number

nig = range(a_int.size(1))  ## ni group
njg = range(a_int.size(2))  ## nj group

icg = range(int(w_int.size(1)))  ## input channel 
ocg = range(int(w_int.size(0)))  ## output channel

ic_tileg = range(int(len(icg)/array_size))
oc_tileg = range(int(len(ocg)/array_size))

kijg = range(w_int.size(2))
ki_dim = int(math.sqrt(w_int.size(2)))  ## Kernel's 1 dim size

######## Padding before Convolution #######
a_pad = torch.zeros(len(icg), len(nig)+padding*2, len(nig)+padding*2).cuda()
# a_pad.size() = [64, 32+2pad, 32+2pad]
a_pad[ :, padding:padding+len(nig), padding:padding+len(njg)] = a_int.cuda()
a_pad = torch.reshape(a_pad, (a_pad.size(0), -1))
# a_pad.size() = [64, (32+2pad)*(32+2pad)]


a_tile = torch.zeros(len(ic_tileg), array_size,    a_pad.size(1)).cuda() 
w_tile = torch.zeros(len(oc_tileg)*len(ic_tileg), array_size, array_size, len(kijg)).cuda() 

for ic_tile in ic_tileg:
    a_tile[ic_tile,:,:] = a_pad[ic_tile*array_size:(ic_tile+1)*array_size,:]

for ic_tile in ic_tileg:
    for oc_tile in oc_tileg:
        w_tile[oc_tile*len(oc_tileg) + ic_tile,:,:,:] = w_int[oc_tile*array_size:(oc_tile+1)*array_size, ic_tile*array_size:(ic_tile+1)*array_size, :]



###########################################

p_nijg = range(a_pad.size(1)) ## psum nij group

psum = torch.zeros(len(ic_tileg), len(oc_tileg), array_size, len(p_nijg), len(kijg)).cuda()
# psum = torch.zeros(16, 4*4).cuda()

for kij in kijg:
    for ic_tile in ic_tileg:
        for oc_tile in oc_tileg:
            for nij in p_nijg:
                psum[ic_tile, oc_tile, :, nij, kij] = \
                torch.matmul(
                    w_tile[len(oc_tileg)*oc_tile + ic_tile,:,:,kij].float(),
                    a_tile[ic_tile,:,nij].float()
                )


w_int shape: torch.Size([16, 16, 9])


In [10]:
'''
activation.txt code (2-bit)
'''
act_bits = 2

a_int = act_int[0, :, :, :]        # now shape [16, 4, 4]
act_tile = a_int
C, H, W = act_tile.shape           # C=16, H=4, W=4

with open("2bit_activation.txt", "w") as f:
    for i in range(H):
        for j in range(W):
            vals = []
            for ch in range(C):  # 16 channels
                val = int(act_tile[ch, i, j].item())
                val = max(0, min(val, (1<<act_bits)-1))   # clip to 0–3
                vals.append(val)

            # pack into 32 bits now
            word = 0
            for v in vals:
                word = (word << act_bits) | v

            bits32 = f"{word:032b}"
            f.write(bits32 + "\n")

In [11]:
import torch
import torch.nn.functional as F

# === Choose layer–27 tile ===
a_full = act_int[0,:,:,:]       # [16,4,4]
w_full = weight_int[:,:,:,:]    # [16,16,3,3]

# Use whole 4x4 as tile
a_tile = a_full                 # [16,4,4]

# Conv wants [N,C,H,W]
a_tile_batch = a_tile.unsqueeze(0).float()  # [1,16,4,4]
w_full_f     = w_full.float()               # [16,16,3,3]

# === Conv: stride=1, padding=1 => keep size 4x4 ===
tile_out = F.conv2d(
    a_tile_batch,
    w_full_f,
    bias=None,
    stride=1,
    padding=1
)   # [1,16,4,4]

# === ReLU ===
tile_out_relu = torch.clamp(tile_out, min=0.0)   # [1,16,4,4]

# === Convert to integer psum ===
tile_out_int = torch.round(tile_out_relu).to(torch.int32)
tile_out_int = tile_out_int[0]   # [16,4,4]

print("tile_out_int shape:", tile_out_int.shape)

# === Write to txt: each line = 256 bits ===
out_path = "2bit_output_relu.txt"

with open(out_path, "w") as f:
    for h in range(tile_out_int.size(1)):     # 4
        for w in range(tile_out_int.size(2)): # 4
            vals = tile_out_int[:, h, w]      # [16]

            bits_per_pixel = []
            for v in vals:
                v16 = int(v.item()) & 0xFFFF
                bits16 = format(v16, "016b")
                bits_per_pixel.append(bits16)

            line = "".join(bits_per_pixel)    # 16×16 = 256 bits
            f.write(line + "\n")

print("Wrote ReLU outputs to", out_path)

tile_out_int shape: torch.Size([16, 4, 4])
Wrote ReLU outputs to 2bit_output_relu.txt


In [12]:
'''
Generate weight_kij files for layer 27 (16→16)
weight_k0.txt ... weight_k8.txt
Each file: 16 rows, each row = 16 weights (4-bit)
packed into 64 bits
'''

import os

w_bit = 4

# weight_int shape now [16,16,3,3], flatten to [16,16,9]
w_int = weight_int.view(weight_int.size(0), weight_int.size(1), -1).int()

os.makedirs("weight_kij", exist_ok=True)

num_kij = w_int.size(2)   # should now be 9
print("num_kij =", num_kij)

for kij in range(num_kij):
    fname = f"weight_kij/weight_k{kij}.txt"
    with open(fname, "w") as f:
        for oc in range(w_int.size(0)):   # 0..15
            vals = []
            for ic in range(w_int.size(1)):   # 0..15
                v = int(w_int[oc, ic, kij].item())
                if v < 0:
                    v += (1 << w_bit)
                v &= (1 << w_bit) - 1
                vals.append(v)

            # pack into 64-bit
            word = 0
            for v in vals:
                word = (word << w_bit) | v

            f.write(f"{word:064b}\n")

    print(f"Saved {fname}")

num_kij = 9
Saved weight_kij/weight_k0.txt
Saved weight_kij/weight_k1.txt
Saved weight_kij/weight_k2.txt
Saved weight_kij/weight_k3.txt
Saved weight_kij/weight_k4.txt
Saved weight_kij/weight_k5.txt
Saved weight_kij/weight_k6.txt
Saved weight_kij/weight_k7.txt
Saved weight_kij/weight_k8.txt


In [13]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# ========= 1. Quantization parameters ============
w_bit = 4   # 4-bit weights
act_bit = 2 # 2-bit activations

# === Weights for layer 27 ===
weight_q = model.features[27].weight_q.detach()      # [16,16,3,3]
w_alpha = model.features[27].weight_quant.wgt_alpha.detach()
w_delta = w_alpha / (2**(w_bit-1) - 1)
w_int = weight_q / w_delta                          # [16,16,3,3]

# === Activations input to layer 27 ===
act27_in = act_int[0].to(device)                    # [16,4,4]
a_full = act27_in.unsqueeze(0)                      # [1,16,4,4]

act_alpha = model.features[27].act_alpha.detach()
a_delta = act_alpha / (2**act_bit - 1)              # scale
# DO NOT re–quantize, `act_int` was already quantized

print("a_int shape:", a_full.shape)
print("w_int shape:", w_int.shape)

# ========= 2. Integer Conv2d =====================
conv2 = nn.Conv2d(
    in_channels = 16,
    out_channels = 16,
    kernel_size = 3,
    padding = 1,
    bias = False,
).to(device)

conv2.weight = nn.Parameter(w_int.float().to(device))

# ========= 3. Integer psum then ReLU ===============
with torch.no_grad():
    psum_int = conv2(a_full.float())    # [1,16,4,4]
    psum_int = torch.clamp(psum_int, min=0)
    psum_recovered = psum_int * w_delta * a_delta

print("psum_recovered shape:", psum_recovered.shape)

# ========= 4. Compute true reference (conv27→relu28) ==
with torch.no_grad():
    y = model.features[27](a_full)  # conv27
    y = model.features[28](y)       # relu28
    y_ref = y[0]                    # [16,4,4]

# ========= 5. Get true next layer input ============
# From SaveOutput:
# k = 8 for layer 27 input
# k+1 = 9 for layer 29 input (after ReLU)
act29_in = save_output.outputs[9][0]  # [128,16,4,4]
act29_in = act29_in[0].to(device)     # [16,4,4]

# ========= 6. Compute errors ========================
err_ref  = (psum_recovered[0] - y_ref).abs().max()
err_next = (psum_recovered[0] - act29_in).abs().max()

print("max |psum_recovered − conv27+relu28|:", err_ref.item())
print("max |psum_recovered − act29_in| :", err_next.item())


a_int shape: torch.Size([1, 16, 4, 4])
w_int shape: torch.Size([16, 16, 3, 3])
psum_recovered shape: torch.Size([1, 16, 4, 4])
max |psum_recovered − conv27+relu28|: 2.843454122543335
max |psum_recovered − act29_in| : 1.7434358596801758e-06


In [15]:
import torch
import os

# ---------- Helpers ----------

def load_activation_txt(path, C=16, H=4, W=4, act_bits=2):
    """Reverse of your activation writer:
       word = 0; for v in vals: word = (word<<act_bits) | v;
       bits = f"{word:032b}"  (MSB = ch0).
    """
    act_rec = torch.zeros(C, H, W, dtype=torch.int32)
    with open(path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    assert len(lines) == H*W, f"activation lines {len(lines)} != {H*W}"

    idx = 0
    mask = (1 << act_bits) - 1
    for i in range(H):
        for j in range(W):
            bits = lines[idx]
            idx += 1
            assert len(bits) == C*act_bits, f"act line len {len(bits)} != {C*act_bits}"
            word = int(bits, 2)

            # unpack ch15..ch0 (reverse of write)
            for ch in reversed(range(C)):
                v = word & mask
                word >>= act_bits
                act_rec[ch, i, j] = v

    return act_rec


def load_weights_kij(folder, C_in=16, C_out=16, w_bit=4, num_kij=9):
    """Reverse of weight writer:
       vals(ic0..ic15), then:
       word = 0; for v in vals: word = (word<<w_bit) | v;
       bits64 = f"{word:064b}".
    """
    w_rec = torch.zeros(C_out, C_in, num_kij, dtype=torch.int32)
    mask = (1 << w_bit) - 1
    sign_bit = 1 << (w_bit - 1)

    for kij in range(num_kij):
        path = os.path.join(folder, f"weight_k{kij}.txt")
        with open(path, "r") as f:
            lines = [line.strip() for line in f if line.strip()]

        assert len(lines) == C_out, f"k={kij}: rows {len(lines)} != {C_out}"

        for oc, bits in enumerate(lines):
            assert len(bits) == C_in * w_bit, f"k={kij}, oc={oc}: len {len(bits)}"
            word = int(bits, 2)

            # unpack ic15..ic0
            for ic in reversed(range(C_in)):
                v = word & mask
                word >>= w_bit
                # back to signed 4-bit two's complement
                if v & sign_bit:
                    v -= (1 << w_bit)
                w_rec[oc, ic, kij] = v

    return w_rec


def load_output_relu(path, C=16, H=4, W=4):
    """Reverse of your output writer:
       for v in vals (ch0..ch15):
           bits16 = f"{v16:016b}"
       line = ''.join(bits16_list)
    """
    out_rec = torch.zeros(C, H, W, dtype=torch.int32)
    with open(path, "r") as f:
        lines = [line.strip() for line in f if line.strip()]

    assert len(lines) == H*W, f"output lines {len(lines)} != {H*W}"

    idx = 0
    for i in range(H):
        for j in range(W):
            bits = lines[idx]
            idx += 1
            assert len(bits) == 16 * C, f"out line len {len(bits)} != {16*C}"

            for ch in range(C):
                seg = bits[16*ch : 16*(ch+1)]   # ch0 first, same as write
                v = int(seg, 2)
                # if you treat psums as signed 16-bit:
                if v >= 0x8000:
                    v -= 0x10000
                out_rec[ch, i, j] = v

    return out_rec


# ---------- 1) Check activation.txt ----------

a_ref = act_int[0].cpu().int()          # [16,4,4] – quantized activations you used
act_rec = load_activation_txt("2bit_activation.txt", C=16, H=4, W=4, act_bits=2)

diff_act = (act_rec - a_ref).abs().max()
print("max |activation.txt - act_int[0]| =", diff_act.item())


# ---------- 2) Check weight_k*.txt ----------

# flatten ki,kj -> kij to match the files
w_ref = weight_int.view(weight_int.size(0), weight_int.size(1), -1).cpu().int()  # [16,16,9]
num_kij = w_ref.size(2)

w_rec = load_weights_kij("weight_kij", C_in=16, C_out=16, w_bit=4, num_kij=num_kij)

diff_w = (w_rec - w_ref).abs().max()
print("max |weight_k*.txt - weight_int|   =", diff_w.item())


# ---------- 3) Check output_relu_16ch.txt ----------

out_ref = tile_out_int.cpu().int()      # [16,4,4] – psums you wrote out
out_rec = load_output_relu("2bit_output_relu.txt", C=16, H=4, W=4)

diff_out = (out_rec - out_ref).abs().max()
print("max |output_relu_16ch.txt - tile_out_int| =", diff_out.item())


max |activation.txt - act_int[0]| = 0
max |weight_k*.txt - weight_int|   = 0
max |output_relu_16ch.txt - tile_out_int| = 0
